Make sure you are familiar with Chapters 1 & 2 of Nathan George's book [Practical Data Science with
Python](https://www.packtpub.com/product/practical-data-science-with-python/9781801071970).
 before diving into this notebook.

To get started, first we need to download the [City of Montreal 311 data
set](https://data.montreal.ca/dataset/5866f832-676d-4b07-be6a-e99c21eb17e4/resource/2cfa0e06-9be4-49a6-b7f1-ee9f2363a872/download/requetes311.csv). We could of course download it dynamically each time using the URL, but let's be nice to the people who host it and save a local copy as `requetes311.csv`. We can use Python to check if that file exists:

In [1]:
import os.path # load a library
filename = 'requetes311.csv' # put the filename in a string variable
if os.path.isfile(filename): # check if the file exists
    print('We have the data at hand.') # what to print out if it does
else: # and what to do if not
    print('Please download the data and put it in this same directory')

We have the data at hand.


Now, we load the data into a Pandas data frame and see what we got in terms of rows and columns. Since this is a rather large file, this might take a while. Be patient.

In [6]:
import pandas as pd # for a local installation, use pip to get pandas
print('Attempting to load the data. (Do NOT click run again.)')
data = pd.read_csv(filename) # load CSV-formatted data from a file (Chapter 3 shows other ways to read dara)
rows, cols = data.shape # consult the shape of the resulting data
print(f'There are {rows} and {cols} columns')
cells = rows * cols
assert cells == data.size # we can check this directly, too

Attempting to load the data. (Do NOT click run again.)
There are 4635093 and 29 columns


What do those columns store? Let's take a look at the *header* of the data frame that by default is the first row of the CSV file (see Chapter 3 for more information and also what to do if the file is in Excel format instead).

In [9]:
for (c, i) in zip(data.columns, range(cols)): # iterate over the contents of the header with a for loop
    print(f'Column at index {i} is called {c}')

Column at index 0 is called ID_UNIQUE
Column at index 1 is called NATURE
Column at index 2 is called ACTI_NOM
Column at index 3 is called TYPE_LIEU_INTERV
Column at index 4 is called RUE
Column at index 5 is called RUE_INTERSECTION1
Column at index 6 is called RUE_INTERSECTION2
Column at index 7 is called LOC_ERREUR_GDT
Column at index 8 is called ARRONDISSEMENT
Column at index 9 is called ARRONDISSEMENT_GEO
Column at index 10 is called LIN_CODE_POSTAL
Column at index 11 is called DDS_DATE_CREATION
Column at index 12 is called PROVENANCE_ORIGINALE
Column at index 13 is called PROVENANCE_TELEPHONE
Column at index 14 is called PROVENANCE_COURRIEL
Column at index 15 is called PROVENANCE_PERSONNE
Column at index 16 is called PROVENANCE_COURRIER
Column at index 17 is called PROVENANCE_TELECOPIEUR
Column at index 18 is called PROVENANCE_INSTANCE
Column at index 19 is called PROVENANCE_MOBILE
Column at index 20 is called PROVENANCE_MEDIASOCIAUX
Column at index 21 is called PROVENANCE_SITEINTE

Oh, look, those are in French. That's cool and also to be expected. The first one sounds a bit boring since it is just a unique ID. It would be nice to know how many unique values each column contains.

In [11]:
counts = data.nunique(axis = 0)
print('This will also take a long time.')
for (c, u) in zip(data.columns, counts): # zero means iterate over rows, one would mean to iterate over columns
    print(f'Column <{c}> contains {u} unique values')

This will also take a long time.
Column <ID_UNIQUE> contains 2279791 unique values
Column <NATURE> contains 4 unique values
Column <ACTI_NOM> contains 1793 unique values
Column <TYPE_LIEU_INTERV> contains 4 unique values
Column <RUE> contains 4808 unique values
Column <RUE_INTERSECTION1> contains 4510 unique values
Column <RUE_INTERSECTION2> contains 4606 unique values
Column <LOC_ERREUR_GDT> contains 2 unique values
Column <ARRONDISSEMENT> contains 36 unique values
Column <ARRONDISSEMENT_GEO> contains 34 unique values
Column <LIN_CODE_POSTAL> contains 34785 unique values
Column <DDS_DATE_CREATION> contains 4448098 unique values
Column <PROVENANCE_ORIGINALE> contains 9 unique values
Column <PROVENANCE_TELEPHONE> contains 47 unique values
Column <PROVENANCE_COURRIEL> contains 19 unique values
Column <PROVENANCE_PERSONNE> contains 20 unique values
Column <PROVENANCE_COURRIER> contains 7 unique values
Column <PROVENANCE_TELECOPIEUR> contains 6 unique values
Column <PROVENANCE_INSTANCE> co

We have already learned something! There are less unique IDs than there are rows in the data, some of the rows appear to be follow-ups to earlier rows. This is vital information if we want to know how many different incidents are reported in the data set since we can no longer assume that each row refers to a distinct incident. To understand more of what is happening, let's start by looking at the columns that only have a handful of unique values. Let's filter out which columns have less than ten unique values:

In [18]:
few = [ data.columns[i] for i in range(cols) if counts[i] < 10 ]
for c in few: # take a look at the values
    for v in data[c].unique():
        print(f'Column <{c}> can take on the value "{v}"')
    print('') # just a blank line between columns

Column <NATURE> can take on the value "Information"
Column <NATURE> can take on the value "Commentaire"
Column <NATURE> can take on the value "Requete"
Column <NATURE> can take on the value "Plainte"

Column <TYPE_LIEU_INTERV> can take on the value "nan"
Column <TYPE_LIEU_INTERV> can take on the value "Adresse"
Column <TYPE_LIEU_INTERV> can take on the value "Troncon"
Column <TYPE_LIEU_INTERV> can take on the value "Intersection"
Column <TYPE_LIEU_INTERV> can take on the value "Installation"

Column <LOC_ERREUR_GDT> can take on the value "nan"
Column <LOC_ERREUR_GDT> can take on the value "1.0"
Column <LOC_ERREUR_GDT> can take on the value "0.0"

Column <PROVENANCE_ORIGINALE> can take on the value "Téléphone"
Column <PROVENANCE_ORIGINALE> can take on the value "Personne"
Column <PROVENANCE_ORIGINALE> can take on the value "Courriel"
Column <PROVENANCE_ORIGINALE> can take on the value "Télécopieur"
Column <PROVENANCE_ORIGINALE> can take on the value "Courrier"
Column <PROVENANCE_ORIGINA

It might be good to take a look at the first few rows now before going any further as well as a few of the last rows.

In [22]:
k = 4 # how many we want to see
print('FIRST')
data.head(k)

FIRST


,ID_UNIQUE,NATURE,ACTI_NOM,TYPE_LIEU_INTERV,RUE,RUE_INTERSECTION1,RUE_INTERSECTION2,LOC_ERREUR_GDT,ARRONDISSEMENT,ARRONDISSEMENT_GEO,...,PROVENANCE_MOBILE,PROVENANCE_MEDIASOCIAUX,PROVENANCE_SITEINTERNET,UNITE_RESP_PARENT,LOC_LONG,LOC_LAT,LOC_X,LOC_Y,DERNIER_STATUT,DATE_DERNIER_STATUT
0,NaN,Information,Feux de circulation - Entretien,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,SERVICE DE L EXPÉRIENCE CITOYENNE ET DES COMMU...,NaN,NaN,NaN,NaN,NaN,2019-11-10T08:48:01
1,NaN,Information,IRP - Fermeture de rue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,DIRECTION GÉNÉRALE,NaN,NaN,NaN,NaN,NaN,2018-01-21T20:04:02
2,NaN,Information,Nid-de-poule,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,DIRECTION GÉNÉRALE,NaN,NaN,NaN,NaN,NaN,2018-01-21T20:26:01
3,NaN,Information,Eau - Divers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,DIRECTION GÉNÉRALE,NaN,NaN,NaN,NaN,NaN,2018-01-22T00:03:17


In [21]:
print('LAST')
data.tail(k)

LAST


,ID_UNIQUE,NATURE,ACTI_NOM,TYPE_LIEU_INTERV,RUE,RUE_INTERSECTION1,RUE_INTERSECTION2,LOC_ERREUR_GDT,ARRONDISSEMENT,ARRONDISSEMENT_GEO,...,PROVENANCE_MOBILE,PROVENANCE_MEDIASOCIAUX,PROVENANCE_SITEINTERNET,UNITE_RESP_PARENT,LOC_LONG,LOC_LAT,LOC_X,LOC_Y,DERNIER_STATUT,DATE_DERNIER_STATUT
4635089,13-313550-D,Requete,Fuite d'eau,Troncon,rue de Port-Royal,rue Meunier,avenue de l'Esplanade,0.0,Ahuntsic - Cartierville,Ahuntsic-Cartierville,...,0,0,0,AHUNTSIC-CARTIERVILLE,-73.659221,45.542706,292366.246000,5.044767e+06,Prise en charge,2020-07-22T06:51:11
4635090,13-313550-C,Requete,Fuite d'eau,Troncon,rue de Port-Royal,rue Meunier,avenue de l'Esplanade,0.0,Ahuntsic - Cartierville,Ahuntsic-Cartierville,...,0,0,0,AHUNTSIC-CARTIERVILLE,-73.659221,45.542706,292366.246000,5.044767e+06,Terminée,2018-05-03T08:06:30
4635091,13-313550-B,Requete,Fuite d'eau,Troncon,rue de Port-Royal,rue Meunier,avenue de l'Esplanade,0.0,Ahuntsic - Cartierville,Ahuntsic-Cartierville,...,0,0,0,AHUNTSIC-CARTIERVILLE,-73.659221,45.542706,292366.246000,5.044767e+06,Terminée,2018-01-17T10:37:13
4635092,10-182618-A,Requete,Trottoir ou bordure - Réparation,Adresse,rue De Castelnau,NaN,NaN,0.0,Villeray-Saint-Michel - Parc-Extension,Villeray-Saint-Michel-Parc-Extension,...,0,0,0,VILLERAY - ST-MICHEL - PARC-EXTENSION,-73.619035,45.536318,295503.353734,5.044052e+06,Terminée,2018-11-01T06:51:40
